In [1]:
##importing libraries
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
#from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
from numba import cuda
from torch.nn import functional as F 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#DEVICE = "cpu"
print(DEVICE)
image_dir = "/content/drive/MyDrive/DRIVE-Seg/DRIVE-Seg/train/image"
label_dir = "/content/drive/MyDrive/DRIVE-Seg/DRIVE-Seg/train/label"
t_image_dir="/content/drive/MyDrive/DRIVE-Seg/DRIVE-Seg/test/image"
t_label_dir="/content/drive/MyDrive/DRIVE-Seg/DRIVE-Seg/test/label"


cuda


In [ ]:
class Devseg_data(Dataset):
    def __init__(self, image_dir, label_dir, transform = None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.labels = os.listdir(label_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir,self.images[index])
        label_path = os.path.join(self.label_dir, self.labels[index])
        image = Image.open(img_path)
        label  = Image.open(label_path)
        
        if self.transform is not None:
           image = self.transform(image)
           label = self.transform(label)
            
        return image, label

In [ ]:
class Devseg_data1(Dataset):
    def __init__(self, t_image_dir, t_label_dir, transform = None):
        self.t_image_dir = t_image_dir
        self.t_label_dir = t_label_dir
        self.transform = transform
        self.images = os.listdir(t_image_dir)
        self.labels = os.listdir(t_label_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        t_img_path = os.path.join(self.t_image_dir,self.images[index])
        t_label_path = os.path.join(self.t_label_dir, self.labels[index])
        image = Image.open(t_img_path)
        #image = np.rollaxis(image,2,0)
        label  = Image.open(t_label_path)
        
        if self.transform is not None:
            image = self.transform(image)
            label = self.transform(label)
            
        return image, label

In [ ]:
def get_loaders(
    image_dir,
    label_dir,
    t_image_dir,
    t_label_dir,
    batch_size,
    transform,
    num_workers = 0,
    pin_memory = True):

    dataset = Devseg_data(
        image_dir = image_dir,
        label_dir=label_dir,
        transform=transform)
    t_dataset = Devseg_data1(
        t_image_dir = t_image_dir,
        t_label_dir=t_label_dir,
        transform=transform)
    train_size = int(1.0*len(dataset))
    val_size = int(1.0*len(t_dataset))
    print(f"Train size = {train_size}")
    print(f"Val size = {val_size}")
    train_ds=train_size
    val_ds=val_size
    train_loader = DataLoader(
            dataset,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory,
            )
    
    val_loader = DataLoader(
            t_dataset,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory,
            )
    
    return dataset,t_dataset,train_loader, val_loader

In [ ]:
class SegNet(nn.Module):
	def __init__(self):
		super(SegNet, self).__init__()

		#SegNet Architecture
		#Takes input of size in_chn = 3 (RGB images have 3 channels)
		#Outputs size label_chn =1

		#ENCODING consists of 5 stages
		#Stage 1, 2 has 2 layers of Convolution + Batch Normalization + Max Pool respectively
		#Stage 3, 4, 5 has 3 layers of Convolution + Batch Normalization + Max Pool respectively

		#General Max Pool 2D for ENCODING layers
		#Pooling indices are stored for Upsampling in DECODING layers
		self.in_chn =3
		self.out_chn = 1

		self.MaxEn = nn.MaxPool2d(2, stride=2, return_indices=True)
		batchNorm_momentum = 0.1
		

		self.ConvEn11 = nn.Conv2d(self.in_chn, 64, kernel_size=3, padding=1)
		self.BNEn11 = nn.BatchNorm2d(64, momentum=batchNorm_momentum)
		self.ConvEn12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
		self.BNEn12 = nn.BatchNorm2d(64, momentum=batchNorm_momentum)

		self.ConvEn21 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
		self.BNEn21 = nn.BatchNorm2d(128, momentum=batchNorm_momentum)
		self.ConvEn22 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
		self.BNEn22 = nn.BatchNorm2d(128, momentum=batchNorm_momentum)

		self.ConvEn31 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
		self.BNEn31 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)
		self.ConvEn32 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
		self.BNEn32 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)
		self.ConvEn33 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
		self.BNEn33 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)

		self.ConvEn41 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
		self.BNEn41 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvEn42 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNEn42 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvEn43 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNEn43 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)

		self.ConvEn51 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNEn51 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvEn52 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNEn52 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvEn53 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNEn53 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)


		#DECODING consists of 5 stages
		#Each stage corresponds to their respective counterparts in ENCODING

		#General Max Pool 2D/Upsampling for DECODING layers
		self.MaxDe = nn.MaxUnpool2d(2, stride=2) 

		self.ConvDe53 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNDe53 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvDe52 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNDe52 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvDe51 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNDe51 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)

		self.ConvDe43 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNDe43 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvDe42 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
		self.BNDe42 = nn.BatchNorm2d(512, momentum=batchNorm_momentum)
		self.ConvDe41 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
		self.BNDe41 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)

		self.ConvDe33 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
		self.BNDe33 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)
		self.ConvDe32 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
		self.BNDe32 = nn.BatchNorm2d(256, momentum=batchNorm_momentum)
		self.ConvDe31 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
		self.BNDe31 = nn.BatchNorm2d(128, momentum=batchNorm_momentum)

		self.ConvDe22 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
		self.BNDe22 = nn.BatchNorm2d(128, momentum=batchNorm_momentum)
		self.ConvDe21 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
		self.BNDe21 = nn.BatchNorm2d(64, momentum=batchNorm_momentum)

		self.ConvDe12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
		self.BNDe12 = nn.BatchNorm2d(64, momentum=batchNorm_momentum)
		self.ConvDe11 = nn.Conv2d(64, self.out_chn, kernel_size=3, padding=1)

	def forward(self, x):

		#ENCODE LAYERS
		#Stage 1
		x = F.relu(self.BNEn11(self.ConvEn11(x))) 
		x = F.relu(self.BNEn12(self.ConvEn12(x))) 
		x, ind1 = self.MaxEn(x)
		size1 = x.size()

		#Stage 2
		x = F.relu(self.BNEn21(self.ConvEn21(x))) 
		x = F.relu(self.BNEn22(self.ConvEn22(x))) 
		x, ind2 = self.MaxEn(x)
		size2 = x.size()

		#Stage 3
		x = F.relu(self.BNEn31(self.ConvEn31(x))) 
		x = F.relu(self.BNEn32(self.ConvEn32(x))) 
		x = F.relu(self.BNEn33(self.ConvEn33(x))) 	
		x, ind3 = self.MaxEn(x)
		size3 = x.size()

		#Stage 4
		x = F.relu(self.BNEn41(self.ConvEn41(x))) 
		x = F.relu(self.BNEn42(self.ConvEn42(x))) 
		x = F.relu(self.BNEn43(self.ConvEn43(x))) 	
		x, ind4 = self.MaxEn(x)
		size4 = x.size()

		#Stage 5
		x = F.relu(self.BNEn51(self.ConvEn51(x))) 
		x = F.relu(self.BNEn52(self.ConvEn52(x))) 
		x = F.relu(self.BNEn53(self.ConvEn53(x))) 	
		x, ind5 = self.MaxEn(x)
		size5 = x.size()

		#DECODE LAYERS
		#Stage 5
		x = self.MaxDe(x, ind5, output_size=size4)
		x = F.relu(self.BNDe53(self.ConvDe53(x)))
		x = F.relu(self.BNDe52(self.ConvDe52(x)))
		x = F.relu(self.BNDe51(self.ConvDe51(x)))

		#Stage 4
		x = self.MaxDe(x, ind4, output_size=size3)
		x = F.relu(self.BNDe43(self.ConvDe43(x)))
		x = F.relu(self.BNDe42(self.ConvDe42(x)))
		x = F.relu(self.BNDe41(self.ConvDe41(x)))

		#Stage 3
		x = self.MaxDe(x, ind3, output_size=size2)
		x = F.relu(self.BNDe33(self.ConvDe33(x)))
		x = F.relu(self.BNDe32(self.ConvDe32(x)))
		x = F.relu(self.BNDe31(self.ConvDe31(x)))

		#Stage 2
		x = self.MaxDe(x, ind2, output_size=size1)
		x = F.relu(self.BNDe22(self.ConvDe22(x)))
		x = F.relu(self.BNDe21(self.ConvDe21(x)))

		#Stage 1
		x = self.MaxDe(x, ind1)
		x = F.relu(self.BNDe12(self.ConvDe12(x)))
		x = self.ConvDe11(x)

		return x

In [ ]:
def check_accuracy(loader, model, device = DEVICE):
    accuracy_array=[]
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device).float()
            y = y.to(device).float()
            preds = (model(x))
            preds = (preds>50).float()
            num_correct += torch.sum((preds ==y)*1.0)
            num_pixels += torch.numel(preds)
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.8f}")
    accuracy_array.append((100*num_correct/num_pixels).item())
    print(f"Accuracy = {accuracy_array}")      
    

In [ ]:
def dice_loss(input, target):
    smooth = 1

    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    print(f"Dice score:{1 - ((2. * intersection + smooth)/(iflat.sum() + tflat.sum() + smooth)):.8f}");
    return 1 - ((2. * intersection + smooth)/(iflat.sum() + tflat.sum() + smooth));
    #print(f"Dice loss:{1 - ((2. * intersection + smooth)/(iflat.sum() + tflat.sum() + smooth)):.8f}");

In [ ]:
def train_fn(loader, model, optimizer,criterion):
  num_epochs=5
  accuracy_array=[]
  for epoch in range(num_epochs):
    loop = tqdm(loader)
    for batch_idx, (data, targets) in enumerate(loop):
          data = data.float().to(device=DEVICE)
          targets = torch.squeeze(((targets>0.5)*1).to(device=DEVICE))

          # forward
          with torch.cuda.amp.autocast():
              predictions = model(data)
              o = torch.cat((predictions,predictions),1)
              loss1 = criterion(o, targets)
              d=dice_loss(predictions, targets)
              #print(f'len:,{targets.size()}:.8f');
              #print(f'length:,{predictions.size()}:.8f')
              #print(f'lengthhh:,{o.size()}:.8f')
              loss=loss1+d
          optimizer.zero_grad()
          #loss=loss1+d;
          loss.backward()
          optimizer.step()

          loop.set_postfix(loss=loss.item())

In [ ]:
def main():
  transform=transforms.Compose([transforms.Resize((584,564)),transforms.ToTensor()])
  learning_rate = 0.001
  num_epochs = 5
  BATCH_SIZE=3
  NUM_WORKERS=0;
  PIN_MEMORY=False
  model = SegNet().cuda()
  model.train()
  criterion = nn.NLLLoss()

  optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
  a,b,train_loader, val_loader = get_loaders(
        image_dir,
        label_dir,
        t_image_dir,
        t_label_dir,
        BATCH_SIZE,
        transform,
        NUM_WORKERS,
        PIN_MEMORY)
 # check_accuracy(val_loader,model,device=DEVICE)
  for epoch in range(num_epochs):
        train_fn(train_loader, model, optimizer,criterion)
        check_accuracy(val_loader,model,device=DEVICE)


In [ ]:

if __name__ == "__main__":
    main() 

Train size = 15
Val size = 5


  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:749: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Dice score:0.99165666


 40%|████      | 2/5 [00:10<00:15,  5.22s/it, loss=0.7]

Dice score:0.88331079
Dice score:0.87927234


...

 80%|████████  | 4/5 [00:01<00:00,  2.70it/s, loss=nan]

Dice score:nan


100%|██████████| 5/5 [00:01<00:00,  2.71it/s, loss=nan]

Dice score:nan


Got 1469263.0/1646880 with acc 89.21493530
Accuracy = [89.2149429321289]
